# Multi-threading, Multi-processing

## 1. 개념 - Process vs Thread

> Process, Thread 

### 1.1. 프로세스

- 운영체제에서 할당 받는 자원의 단위와 실행 중인 프로그램
- cpu동작 시간과 주소공간, 메모리가 독립적 (프로그램 상의 의존이 없음)
- Code, Data, Stack, Heap -> 독립적
- 최소 1개의 메인 스레드를 보유함
- 파이프, 파일, 소켓 등을 사용해서 프로세스 간 통신 (cost가 높음) -> context switching

### 1.2. 스레드

- 프로세스 내에 실행 흐름 단위
- 프로세스 자원 사용
- stack만 별도로 할당하고 나먼지는 공유함(code, data, heap)
- 한 스레드의 결과가 다른 스레드에 영향을 미침
- 디버깅이 어려움. 따라서 동기화에 주의

### 1.3. 멀티 스레드

- 한 개의 단일 어플리케이션(또는 응용프로그램) -> 여러 스레드로 구성 후 작업 처리
- 시스템 자원의 소모가 감소함(효율성), 처리량이 증가함(cost 감소함)
- 통신 부담은 감소
- 디버깅이 어려움
- 단일 프로세스에는 효과가 미약함
- 자원 공유의 문제가 발생할 수 있음 (교착상태 deadlock 발생)
- 프로세스에 영향을 줄 수 있음

### 1.4. 멀티 프로세스

- 한 개의 단일 어플리케이션(또는 응용프로그램) -> 여러 프로세스로 구성 후 작업 처리
- 한 개의 프로세스 문제 발생은 확산이 없음 (프로세스 kill)
- 복잡한 통신 방법을 사용하기 때문에 캐시 체인지, Cost 비용이 매우 높음(오버헤드가 상승할 가능성)

## 2. GIL (Global Interpreter Lock)

> Cpython, 메모리관리, GIL을 사용하는 이유

### 2.1. GIL의 기원

1. GIL을 두어서 I/O bound에 제한을 두고자 함.
    - 단일 스레드만이 python object에 접근하게 제한하는 mutex
2. Cpython 메모리 관리가 취약하기 때문에, thread-safe를 보장하기 위해 GIL을 만듬.
3. 단일 스레드로도 충분히 빠름.
4. 멀티 프로세스를 이미 사용하기 때문에 GIL을 없앨 이유가 없음. (Numpy, Scipy 등은 이미 GIL 외부 영역에서 효율적인 코딩을 가능하게 함.)
5. 병렬처리는 Multiprocessing, asyncio 등 선택지가 다양함.
6. 스레드 동시성 완벽 처리를 위해서는 -> Jython, IronPython, Stackless Python 등이 존재함.

## 3. Thread - basic

### 예제 1.

`snippet_01.py`

- 메인스레드는 서브스레드가 종료되기 전에 이미 실행 후 종료함.
- 메인스레드가 종료되어도, 서브스레드는 마저 실행되고 종료함.


### 예제 2. 

`snippet_02.py`

- `.join()`
- 메인스레드는 서브스레드의 종료까지 대기한 후 종료됨.

## 4. Thread - Daemon, Join

### 4.1. Daemon thread

- 백그라운드에서 실행
- 메인스레드 종료 시, 즉시 강제 종료 함
- 주로 백그라운드 무한 대기 이벤트 발생을 실행하는 부분을 담당함.
    - 예) 
        - JVM(가비지 컬렉션)
        - 워드프로세서: 메인스레드는 작성, 데몬스레드는 자동저장의 기능
- 파이썬에서 `daemon=False`가 디폴트이므로, 데몬스레드를 하고자 할 시, 꼭 옵션을 줘야 함.
        
### 일반 예제 1. 

- `snippet_03.py`: 메인스레드, 스레드y, 스레드x 순으로 종료됨.
- `snippet_04.py` (`daemon=True` 적용): 메인스레드가 종료되면, 나머지는 모두 강제종료됨.

## 5. Multithreading

> Many Thread, `concurrent.futures` , `PoolExecuter`

### 5.1. Many Thread (그룹 스레드)

1. 파이썬 3.2 이상에서 표준 라이브러리.
2. `.concurrent.futures`
3. `.with`의 사용으로 스레드를 생성/소멸의 라이프사이클을 관리하는데 용이함.
4. 디버깅이 난해함.
5. 동작방법
    - 대기중인 작업
    - 큐에 대기 시킴
    - 완료 상태 조사
    - 결과 또는 예외를 받아옴
    - 패키지를 활용해서 캡슐화(단일화) 가능

**snippet**

- `snippet_05.py`: 복수의 스레드를 일일이 생성
- `snippet_06.py`: with를 통해 생성

### 5.2. Lock, Deadlock

> Lock, Deadlock, Race Conditions, Thread Synchronization

**용어 설명**

1. semaphore(세마포어)
    - 프로세스 간 공유된 자원에 접근 시 문제발생 가능하므로
    - 한개의 프로세스만 접근할 수 있게 처리함
    - 경쟁 상태 예방
2. mutex(뮤텍스)
    - 공유된 자원의 데이터를 여러 스레드가 접근하는 것을 막는 것.
    - 경쟁 상태를 예방. 동기화.
3. Lock
    - 상호 배제를 위한 잠금 처리 -> 데이터를 경쟁
4. Deadlock(교착상태)
    - 프로세스가 자원을 획득하지 못해 다음 처리를 하지 못 하는 무한대기 상태
5. Thread Synchronization
    - 스레드 동기화를 통해 안정적으로 동작하게 처리하는 기법
    - 동기화 메소드, 동기화 블럭


**Semaphore and Mutex의 차이점**

- 세마포어와 뮤텍스는 모두 병렬 프로그래밍 환경에서 상호배제를 위해 사용됨.
    - 뮤텍스는 단일 스레드의 접근을 허용.
    - 세마포어는 리소스에 대한 제한된 수의 동시 엑세스를 허용.
        - 따라서, 세마포어는 뮤텍스가 될 수 있지만, 뮤텍스는 세마포어가 될 수 없음.
        
        
**snippet**
- `snippet_07_1.py` : 동기화를 실패한 케이스
- `snippet_07_2.py` : 락을 적용하여 동기화를 구현한 케이스

### 5.3. Producer / Consumer

> 생산자소비자패턴, Producer, Consumer, Queue

**용어 설명**

1. Producer-Consumer Pattern
    - 멀티 스레드 디자인 패턴의 정석
    - 서버 측 프로그래밍의 핵심
    - 허리 역할

**개념**

Python Event 객체 사용 (`threading.Event()`)
1. .Flag의 초기값 = 0
2. .Set() = 1
3. .Clear() = 0
4. .Wait(1 -> 리턴, 0 -> 대기)
5. .isSet() -> 현 플래그 상태

**snippet**

- `snippet_08.py` : Producer-Consumer 패턴 구현

## 2. Multiprocessing

